# Final Project - Part 3 | Model

1. Identify and implement at least three different models.
    - Model choices must include at least one supervised and one unsupervised learning model.
    
2. The business objective can vary for supervised versus unsupervised model.

3. For each model prepare a report showing 
    - results obtained and hyperparameter values used
    - justification for dimensionality reduction (or not) including supporting data
    - resulting conclusions for each model regarding potential fit. 

## Approach

## RNN Timeseries Market Data

## Classification Model - XGBoost?

## Segmentation Model - KMeans